# VAE with Newton's law decoder

In [7]:
import os
import sys

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import gpytorch
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import copy
from torch.utils.data import Dataset, DataLoader,random_split, Subset
from tqdm import tqdm
import random

sys.path.append(os.path.abspath('..'))
from Models.PIVAE import PIVAE
from physics import NewtonsLaw

In [8]:
def set_seed_(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [9]:
# Import RICO data 
file_path = '../Data/RICO4_Dataset_processed.hdf'
df = pd.read_hdf(file_path)
#  define surrounding temperature
df.loc[:, 'sur_temp'] = (df['RTD417'] + df['B.ASTRHT2.T'])/2
df_c = df.copy()
df_c = df[df['temp_change_class'] == 'c']
def select_points(group):
    return group.iloc[::10]  # Select every 10th row
df_c = df_c.groupby('Scheduler Step').apply(select_points).reset_index(drop=True)
df_c = df_c.dropna()

/var/folders/46/30710m094qv2_dhfs08nzzd00000gn/T/ipykernel_23587/3618831305.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_c = df_c.groupby('Scheduler Step').apply(select_points).reset_index(drop=True)


In [10]:
class RICO4Dataset(Dataset):
    def __init__(self, df, select_var):
        self.df = df
        self.select_var = select_var
        self.unique_batches = df['interval'].unique()

    def __len__(self):
        return len(self.unique_batches)

    def __getitem__(self, idx):
        batch = self.unique_batches[idx]
        temp = self.df[self.df['interval'] == batch][self.select_var]
        
        # Normalize and convert to tensors
        ti = torch.tensor(temp['time_within_interval'].to_numpy() / 240, dtype=torch.float32)
        Ts = torch.tensor(temp['sur_temp'].to_numpy(), dtype=torch.float32)
        heat_obs = torch.tensor(temp['B.RTD1'].to_numpy(), dtype=torch.float32)
        
        # Stack the tensors to create the input batch
        x_batch = torch.stack((heat_obs, Ts, ti), dim=1)
        
        return x_batch
# Initialize the dataset and train_loader
select_var = ['time_within_interval', 'sur_temp', 'B.RTD1']
dataset = RICO4Dataset(df_c,select_var)
# Split the dataset into above_20 and below_20
above_20_data = []
below_20_data = []

for idx in range(len(dataset)):
    x_batch = dataset[idx]
    starting_value = x_batch[0, 0].item()  # Get the starting value of B.RTD1
    if starting_value > 20:
        above_20_data.append(x_batch)
    else:
        below_20_data.append(x_batch)

# Define custom datasets for above_20 and below_20
class SubsetDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Create DataLoader instances
below_20_dataset = SubsetDataset(below_20_data)
above_20_dataset = SubsetDataset(above_20_data)

train_loader = DataLoader(above_20_dataset, batch_size=1, shuffle=True)  # Adjust batch_size as needed
val_loader = DataLoader(below_20_dataset, batch_size=1, shuffle=True)

# Check DataLoader sizes
print(f"Number of batches in train_loader: {len(train_loader)}")
print(f"Number of batches in val_loader: {len(val_loader)}")

Number of batches in train_loader: 21
Number of batches in val_loader: 7


In [11]:
set_seed_(123)
model = PIVAE(hidden_layers=[10,10], 
                 activation=nn.Tanh(),
                 mu_prior=torch.tensor(0),
                 var_prior=torch.tensor(2.0))
# Initialize model parameters and optimizer
pars = model.parameters()
opt = torch.optim.Adam(pars, lr=0.0001)

# Lists to store loss values
l_loss, KL, mse = [], [], []

# Set beta value and number of epochs
beta_phy = 1
num_epochs = 1000
l_loss, KL, mse = model.fit(train_loader, opt, beta_phy, num_epochs)


Progress: Epoch 200/1000 | Loss=1.6626, KL=0.1717, MSE=1.4909
Progress: Epoch 400/1000 | Loss=1.7505, KL=0.2124, MSE=1.5380
Progress: Epoch 600/1000 | Loss=1.1644, KL=0.2098, MSE=0.9546
Progress: Epoch 800/1000 | Loss=1.4822, KL=0.2083, MSE=1.2739
Progress: Epoch 1000/1000 | Loss=1.4612, KL=0.2207, MSE=1.2405


In [ ]:
# torch.save(model.state_dict(), 'pre_trained_models/PIVAE_cooling.pth')